### Importing Libraries

In [28]:
#import statements
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.ml.feature import Bucketizer

from pyspark.sql.functions import *

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark.conf.set("spark.sql.caseSensitive", "true")

In [29]:
spark = SparkSession.builder.enableHiveSupport().appName('AmazonData').getOrCreate()
sc = spark.sparkContext

In [30]:
spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.dynamicAllocation.minExecutors', '1'),
 ('spark.driver.appUIAddress',
  'http://bigdataproject-m.us-central1-b.c.sustained-spark-343705.internal:40965'),
 ('spark.executor.memory', '5739m'),
 ('spark.kryoserializer.buffer.max', '2000M'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.driver.maxResultSize', '0'),
 ('spark.yarn.unmanagedAM.enabled', 'true'),
 ('spark.sql.autoBroadcastJoinThreshold', '43m'),
 ('spark.app.id', 'application_1647288588217_0002'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.repl.local.jars',
  'file:///root/.ivy2/jars/com.johnsnowlabs.nlp_spark-nlp_2.12-3.3.4.jar,file:///root/.ivy2/jars/com.typesafe_config-1.4.1.jar,file:///root/.ivy2/jars/org.rocksdb_rocksdbjni-6.5.3.jar,file:///root/.ivy2/jars/com.amazonaws_aws-java-sdk-bundle-1.11.603.jar,file:///root/.ivy2/jars/com.github.universal-automata_liblevenshtein-3.0.0.jar,file:///root/

### Importing Datasets

In [31]:
AF= 'gs://sunenabigdata/actaul_data_json/AMAZON_FASHION.json'
AB= 'gs://sunenabigdata/actaul_data_json/All_Beauty.json'
CSJ= 'gs://sunenabigdata/actaul_data_json/Clothing_Shoes_and_Jewelry.json'
LB= 'gs://sunenabigdata/actaul_data_json/Luxury_Beauty.json'
mAF= 'gs://sunenabigdata/meta_data_json/meta_AMAZON_FASHION.json'
mAB= 'gs://sunenabigdata/meta_data_json/meta_All_Beauty.json'
mCSJ= 'gs://sunenabigdata/meta_data_json/meta_Clothing_Shoes_and_Jewelry.json'
mLB= 'gs://sunenabigdata/meta_data_json/meta_Luxury_Beauty.json'

In [32]:
df_AF = spark.read.json(AF)
df_AF = df_AF.withColumn("Category", lit("Amazon Fashion"))
df_AB = spark.read.json(AB)
df_AB = df_AB.withColumn("Category", lit("All Beauty"))
df_CSJ = spark.read.json(CSJ)
df_CSJ = df_CSJ.withColumn("Category", lit("Clothing, Shoes and Jewelery"))
df_LB= spark.read.json(LB)
df_LB = df_LB.withColumn("Category", lit("Luxury Beauty"))

In [33]:
mdf_AF = spark.read.json(mAF)
mdf_AB = spark.read.json(mAB)
mdf_CSJ = spark.read.json(mCSJ)
mdf_LB= spark.read.json(mLB)

In [34]:
df1_AF=df_AF.drop("reviewerName","style","image")
df1_AB=df_AB.drop("reviewerName","style","image")
df1_CSJ=df_CSJ.drop("reviewerName","style","image")
df1_LB=df_LB.drop("reviewerName","style","image")

whole= df1_AF.union(df1_AB)
whole= whole.union(df1_CSJ)
whole=whole.union(df1_LB)

In [35]:
#renaming asin column to productID, saving in a new dataframe called whole2
whole2 = whole.withColumnRenamed("asin","productID")

In [36]:
mdf1_AF=mdf_AF.drop('also_buy','also_view', 'description', 'details', 'feature', 'fit',
                    'imageURL','imageURLHighRes', 'similar_item','tech1')
mdf1_AB=mdf_AB.drop('also_buy','also_view','category', 'description', 'details', 'feature', 'fit',
                    'imageURL','imageURLHighRes', 'main_cat','similar_item','tech1', 'tech2')
mdf1_CSJ=mdf_CSJ.drop('also_buy','also_view','category', 'description', 'details', 'feature', 'fit',
                    'imageURL','imageURLHighRes', 'main_cat','similar_item','tech1', 'tech2')
mdf1_LB=mdf_LB.drop('also_buy','also_view','category', 'description', 'details', 'feature', 'fit',
                    'imageURL','imageURLHighRes', 'main_cat','similar_item','tech1', 'tech2')

m_whole= mdf1_AF.union(mdf1_AB)
m_whole= m_whole.union(mdf1_CSJ)
m_whole=m_whole.union(mdf1_LB)

In [37]:
finaldf= whole2.join(m_whole, whole2.productID == m_whole.asin, 'inner')
finaldf = finaldf.drop("asin")

In [38]:
finaldf = finaldf.select(concat(finaldf.productID, finaldf.reviewerID, finaldf.unixReviewTime).alias("uniqueID"), "*")

In [39]:
finaldf = finaldf.dropDuplicates(["uniqueID"])

In [40]:
finaldf.count()

33464003

### 10 Core data -  Selecting Only products which have more than 10 reviews

In [41]:
df=finaldf.groupBy(col('productID').alias('productID1')).count().orderBy("count", ascending=False)

In [42]:
df2= finaldf.join(df, finaldf.productID == df.productID1, 'inner')

In [43]:
df2=df2.drop('productID1')

In [44]:
df3=df2.filter(df2['count']>9)

In [45]:
main_final=df3.na.drop(subset=["reviewText"])

In [46]:
df6= main_final.select('overall','reviewText')

In [47]:
df6.show()

+-------+--------------------+
|overall|          reviewText|
+-------+--------------------+
|    5.0|It gives you a re...|
|    5.0|This accessory ki...|
|    3.0|Teeth weren't rea...|
|    5.0|         Great stuff|
|    4.0|the teeth made th...|
|    5.0|Good deal, really...|
|    5.0|Excellent deal fo...|
|    1.0|      Missing items.|
|    1.0|            Garbage!|
|    1.0|            Not good|
|    1.0|Wig is too dark, ...|
|    2.0|Ok product but cheap|
|    4.0|Over all very goo...|
|    4.0|I mostly bought t...|
|    2.0|The wig was a wil...|
|    4.0|The plastic glass...|
|    3.0|The wig was actua...|
|    5.0|Wig was great! Ot...|
|    1.0|When I received t...|
|    1.0|Wig sucks.  The t...|
+-------+--------------------+
only showing top 20 rows



### Converting review text into Term Frequencies

In [48]:
%%time
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

#tokenize words
tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")
df6 = tokenizer.transform(df6)

#drop the redundant source column
df6= df6.drop("reviewText")
df6.show(5)

+-------+--------------------+
|overall|               words|
+-------+--------------------+
|    2.0|[ok, product, but...|
|    1.0|[i, got, this, fo...|
|    3.0|[teeth, weren't, ...|
|    1.0|   [missing, items.]|
|    4.0|[the, teeth, made...|
+-------+--------------------+
only showing top 5 rows

CPU times: user 216 ms, sys: 115 ms, total: 331 ms
Wall time: 2min 3s


In [49]:
%%time
from pyspark.ml.feature import StopWordsRemover

#remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
df6 = remover.transform(df6)

#drop the redundant source column
df6= df6.drop("words")
df6.show(5)

+-------+--------------------+
|overall|            filtered|
+-------+--------------------+
|    2.0|[ok, product, cheap]|
|    3.0|[wig, actually, b...|
|    4.0|[mostly, bought, ...|
|    1.0|[bought, go, cost...|
|    2.0|[wig, wild, mess,...|
+-------+--------------------+
only showing top 5 rows

CPU times: user 266 ms, sys: 105 ms, total: 371 ms
Wall time: 1min 47s


In [50]:
%%time
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(df6)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
nlpdf = idfModel.transform(featurizedData)
nlpdf.select("features").show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                             |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(20,[1,3,5,7,8,9,11,12,13,15,17,18],[1.8057870701235432,2.8108446955451796,2.534706635413931,2.0081336229444,0.6428947985542377,0.7409836504433677,1.925414184194805,2.3424631582315865,0.9339027639750275,3.01

In [51]:
%%time
from pyspark.ml.feature import StringIndexer

nlpdf = nlpdf.withColumn(
'ratings',
F.when((F.col("overall") <= 5.0) & (F.col("overall") > 4.0), "4-5") \
    .when((F.col("overall") <= 4.0) & (F.col("overall") > 3.0), "3-4") \
    .when((F.col("overall") <= 3.0) & (F.col("overall") > 2.0), "2-3") \
    .when((F.col("overall") <= 2.0) & (F.col("overall") > 1.0), "1-2") \
    .when((F.col("overall") <= 1.0) & (F.col("overall") > 0), "0-1") \
)

indexer = StringIndexer(inputCol="ratings", outputCol="label")
indexer_model = indexer.fit(nlpdf)
nlpdf=indexer_model.transform(nlpdf)

CPU times: user 307 ms, sys: 71.8 ms, total: 378 ms
Wall time: 2min 8s


In [52]:
#Splitting the data into test and train data
train, test = nlpdf.randomSplit(weights=[0.8,0.2], seed=200)

### Running Linear Regression, Logistic Regression and Random Forest

In [ ]:
%%time
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Set parameters for Logistic Regression
lgr = LogisticRegression(maxIter=10, featuresCol = 'features', labelCol='label')

# Fit the model to the data.
lgrm = lgr.fit(train)

# Given a dataset, predict each point's label, and show the results.
predictions = lgrm.transform(test)

#print evaluation metrics
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))

0.6043846991720532


0.46058148176334546
CPU times: user 1.77 s, sys: 587 ms, total: 2.35 s
Wall time: 23min 1s


In [ ]:
100